In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
BASE_DIR = Path('C:/Users/atw10wp4/Jupyter/AdventOfCode/Data')

In [3]:
fileName = '2021_13_input.txt'
fileNameFullPath = BASE_DIR / fileName

In [4]:
df = pd.read_csv(fileNameFullPath, header=None, sep=',', names=['x', 'y'])

In [5]:
foldings_df = df[df.y.isna()]['x'].str.split('=', expand=True)
foldings_df[0] = foldings_df[0].str.split(' ', expand=True).iloc[:,-1:]
foldings_df.columns = ['axis', 'fold']
foldings_df = foldings_df.reset_index(drop=True)
foldings_df['fold'] = foldings_df['fold'].astype(int)

In [6]:
dots_df = df[df.y.notna()]
dots_df = dots_df.astype(int)

In [7]:
dots_df['vals'] = np.full(dots_df.shape[0], 1)

In [8]:
xy_matrix = dots_df.pivot(index='y', columns='x', values='vals')
xy_matrix[xy_matrix.isna()] = 0
xy_matrix = xy_matrix.astype(int)
xy_matrix = xy_matrix.reindex(range(0, dots_df.y.max()+1), columns=range(0, dots_df.x.max()+1), fill_value=0)

In [9]:
def fold_up(df, fold):
    upper_fold = df.iloc[:fold,:]
    lower_fold = df.iloc[fold+1:,:].iloc[::-1]
    lower_fold.index = upper_fold.index
    return lower_fold + upper_fold

In [10]:
def fold_left(df, fold):
    left_fold = df.iloc[:,:fold]
    right_fold = df.iloc[:,fold+1:]
    right_fold = right_fold[right_fold.columns[::-1]]
    right_fold.columns = left_fold.columns
    return left_fold + right_fold

In [11]:
for axis, fold in zip(foldings_df['axis'], foldings_df['fold']):
    if axis == 'y':
        xy_matrix = fold_up(xy_matrix, fold)
    if axis == 'x':
        xy_matrix = fold_left(xy_matrix, fold)

In [12]:
display_df = (xy_matrix > 0) * 1

In [13]:
pd.options.display.max_columns = None

In [14]:
display_df.style.background_gradient(vmin=0, vmax=1)

x,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
y,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0
1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
2,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,1,0,0
3,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,0,0,1,0
4,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0
5,0,1,1,0,0,1,1,1,1,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,1,1,0,0
